In [1]:
import pandas as pd
from tqdm import tqdm_notebook as tqdm
import os

pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", -1)
pd.set_option("display.float_format", "{:.4f}".format)
pd.set_option("display.max_info_rows", 200)

In [2]:
import sephora_scrape as scrape
%load_ext autoreload
%autoreload 2

# Brand List

We'll be able to scrape all products from the website by doing it per brand.  First, let's get a list of all brands and the corresponding url link.

In [15]:
sephora = scrape.Sephora()
sephora_brands_url = "https://www.sephora.com/brands-list"
sephora.open_url(sephora_brands_url)
sephora.get_brands()

In [19]:
len(sephora.brand_list), sephora.brand_list[:10]

(338,
 [{'brand_name': 'Acqua di Parma',
   'brand_link': 'https://www.sephora.com/brand/acqua-di-parma'},
  {'brand_name': 'adwoa beauty',
   'brand_link': 'https://www.sephora.com/brand/adwoa-beauty'},
  {'brand_name': 'AERIN',
   'brand_link': 'https://www.sephora.com/brand/aerin-perfume'},
  {'brand_name': 'Aether Beauty',
   'brand_link': 'https://www.sephora.com/brand/aether-beauty'},
  {'brand_name': 'Algenist',
   'brand_link': 'https://www.sephora.com/brand/algenist'},
  {'brand_name': 'Alpha-H',
   'brand_link': 'https://www.sephora.com/brand/alpha-h'},
  {'brand_name': 'alpyn beauty',
   'brand_link': 'https://www.sephora.com/brand/alpyn-beauty'},
  {'brand_name': 'ALTERNA Haircare',
   'brand_link': 'https://www.sephora.com/brand/alterna'},
  {'brand_name': 'amika', 'brand_link': 'https://www.sephora.com/brand/amika'},
  {'brand_name': 'AMOREPACIFIC',
   'brand_link': 'https://www.sephora.com/brand/amorepacific'}])

In [17]:
df_brands = pd.DataFrame(sephora.brand_list)
df_brands.to_csv("brands.csv", index=False)
sephora.browser.quit()

In [3]:
df_brands = pd.read_csv("brands.csv")
df_brands.head()

,brand_name,brand_link
0,Acqua di Parma,https://www.sephora.com/brand/acqua-di-parma
1,adwoa beauty,https://www.sephora.com/brand/adwoa-beauty
2,AERIN,https://www.sephora.com/brand/aerin-perfume
3,Aether Beauty,https://www.sephora.com/brand/aether-beauty
4,Algenist,https://www.sephora.com/brand/algenist


# Product List & Detailed Information

In [4]:
json_folder = "/Users/valmadrid/DataScienceBootcamp/Projects/Gift Recommendation/Gift-Recommendation/web_scraping/json/"
if not os.path.exists(json_folder):
    os.mkdir(json_folder)

In [5]:
images_folder = "/Users/valmadrid/DataScienceBootcamp/Projects/Gift Recommendation/Gift-Recommendation/web_scraping/images/"
if not os.path.exists(images_folder):
    os.mkdir(images_folder)

In [9]:
df_products = pd.DataFrame()

brand = scrape.Sephora()
product = scrape.Sephora()
brand_urls = df_brands.brand_link.unique()

for brand_url in tqdm(brand_urls):

    for page in range(1, 10):

        if page == 1:
            full_url = brand_url + "?pageSize=300"
        else:
            full_url = brand_url + "?pageSize=300&currentPage=" + str(page)

        brand.open_url(full_url)  # open url
        brand.get_productsJSON()  # get products from text/json
        if len(brand.product_list) == 0:
            # if get_productsJSON() returns None then get products class a's
            brand.get_productsHTML()  

        df_products = df_products.append(pd.DataFrame(brand.product_list),
                                         ignore_index=True)
        
#         for i in brand.product_list:
#             product.open_url(i["product_link"])  # open url of product i
#             product.get_product_info(i["product_id"], json_folder) #save json
#             product.get_image(i["product_sku"], images_folder) #save image
        
        if len(brand.product_list) < 300:
            # if current page has less than 300 items, then go to next brand
            break
        else:
            # otherwise go to next page
            continue

    df_products.to_csv("product_list.csv", index=False)

brand.browser.quit()
product.browser.quit()

del brand, product, brand_urls, full_url, brand_url, page, i

In [10]:
df_products

,brand_name,product_id,product_sku,product_name,product_price_low,product_price_high,product_rating,product_image,product_link
0,Acqua di Parma,P460442,2377984,LA DOUBLE J Blu Mediterraneo Mini Set,66.0000,66.0000,5.0000,https://www.sephora.com/productimages/sku/s2377984-main-Lhero.jpg,https://www.sephora.com/product/acqua-di-parma-double-j-blu-mediterraneo-mini-set-P460442
1,Acqua di Parma,P443401,2218774,Blu Mediterraneo MINIATURE Set,66.0000,66.0000,4.2000,https://www.sephora.com/productimages/sku/s2218774-main-Lhero.jpg,https://www.sephora.com/product/blu-mediterraneo-minature-set-P443401
2,Acqua di Parma,P307801,1417609,Fico di Amalfi,30.0000,180.0000,4.5610,https://www.sephora.com/productimages/sku/s1417609-main-Lhero.jpg,https://www.sephora.com/product/blu-mediterraneo-fico-di-amalfi-P307801
3,Acqua di Parma,P307804,1417617,Mirto di Panarea,30.0000,180.0000,4.6800,https://www.sephora.com/productimages/sku/s1417617-main-Lhero.jpg,https://www.sephora.com/product/blu-mediterraneo-mirto-di-panarea-P307804
4,Acqua di Parma,P460441,2379139,Rosa Nobile Shimmering Body Oil,87.0000,87.0000,5.0000,https://www.sephora.com/productimages/sku/s2379139-main-Lhero.jpg,https://www.sephora.com/product/acqua-di-parma-rosa-nobile-shimmering-body-oil-P460441
...,...,...,...,...,...,...,...,...,...
8979,The 7 Virtues,P428712,2047710,Rose Amber Eau de Parfum Rollerball,29.0000,29.0000,4.4667,https://www.sephora.com/productimages/sku/s2047710-main-Lhero.jpg?pb=2020-03-sephora-clean-2019,https://www.sephora.com/product/rose-amber-eau-de-parfum-rollerball-P428712
8980,The 7 Virtues,P428716,2047702,Vetiver Elemi Eau de Parfum Rollerball,29.0000,29.0000,4.5789,https://www.sephora.com/productimages/sku/s2047702-main-Lhero.jpg?pb=2020-03-sephora-clean-2019,https://www.sephora.com/product/vetiver-elemi-eau-de-parfum-rollerball-P428716
8981,8Greens,P452918,2298891,8Greens Gummies Dietary Supplement,45.0000,45.0000,4.0000,https://www.sephora.com/productimages/sku/s2298891-main-Lhero.jpg,https://www.sephora.com/product/8greens-8greens-gummies-dietary-supplement-P452918
8982,8Greens,P437988,2162444,8G Dietary Supplement,14.0000,78.0000,4.4219,https://www.sephora.com/productimages/sku/s2162444-main-Lhero.jpg,https://www.sephora.com/product/8greens-dietary-supplement-P437988


In [ ]:
# product = scrape.Sephora()

# product_links = zip(df_products.product_id.values[:5], df_products.product_link.values[:5])

# for id_, url in tqdm(product_links):

#     product.open_url(url)  # open url
#     product.get_product_info(id_, json_folder)
#     sleep(2)

# product.browser.quit()

# del product, product_links

In [105]:
downloaded_json = [file for file in os.listdir(json_folder) if file.endswith(".json")]
downloaded_json.sort()

# Product Categories & List

In [142]:
categories_urls = [
    "https://www.sephora.com/beauty/new-beauty-products",
    "https://www.sephora.com/shop/gifts-for-her",
    "https://www.sephora.com/shop/gifts-for-men",
    "https://www.sephora.com/shop/gifts-for-them",
    "https://www.sephora.com/shop/gifts-for-teenage-girls",
    "https://www.sephora.com/shop/gift-sets-for-men",
    "https://www.sephora.com/shop/travel-size-toiletries",
    "https://www.sephora.com/shop/value-sets",
    "https://www.sephora.com/shop/editors-picks-gifts?",
    "https://www.sephora.com/shop/luxury-gifts",
    "https://www.sephora.com/shop/exclusive-products",
    "https://www.sephora.com/shop/mens-perfume",
    "https://www.sephora.com/shop/mens-facial-products",
    "https://www.sephora.com/shop/mens-grooming",
    "https://www.sephora.com/shop/mens-hair-care",
    "https://www.sephora.com/shop/mens-personal-care",
    "https://www.sephora.com/beauty/new-cologne-for-men",
    "https://www.sephora.com/beauty/best-selling-mens-products"
]

In [149]:
categories_name = [
    "just_arrived", "gifts_her", "gifts_men", "gifts_them", "gifts_teens",
    "gift_sets_men", "travel_size", "gift_sets", "editors_picks",
    "luxury_gifts", "sephora_exclusives", "perfume_men", "facial_men",
    "grooming_men", "hair_men", "personal_care_men", "just_arrived_men", "best_seller_men"
]

In [157]:
categories = {}

category = scrape.Sephora()

for url, cat in tqdm(zip(categories_urls, categories_name)):
    
    products = []
    
    for page in range(1, 10):

        if page == 1:
            full_url = url + "?pageSize=300"
        else:
            full_url = url + "?pageSize=300&currentPage=" + str(page)

        category.open_url(full_url)  # open url
        category.get_productsJSON()  # get products from text/json
        if len(category.product_list) == 0:
            # if get_productsJSON() returns None then get products class a's
            category.get_productsHTML()  
        
        products.extend([i["product_id"] for i in category.product_list])
        
        if len(category.product_list) < 300:
            # if current page has less than 300 items, then go to next brand
            break
        else:
            # otherwise go to next page
            continue

    categories[cat] = products
    with open("categories.json", "w") as outfile:
        json.dump(categories, outfile, indent=2)
    
category.browser.quit()

del category

[autoreload of sephora_scrape failed: Traceback (most recent call last):
  File "/Applications/anaconda3/envs/learn-env/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/Applications/anaconda3/envs/learn-env/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 450, in superreload
    update_generic(old_obj, new_obj)
  File "/Applications/anaconda3/envs/learn-env/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 387, in update_generic
    update(a, b)
  File "/Applications/anaconda3/envs/learn-env/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 357, in update_class
    update_instances(old, new)
  File "/Applications/anaconda3/envs/learn-env/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 312, in update_instances
    update_instances(old, new, obj.__dict__, visited)
  File "/Applications/anaconda3/envs/learn-env/lib/python3.6/site

# Product Stats & Reviews

In [ ]:
reviews_folder = "/Users/valmadrid/DataScienceBootcamp/Projects/Gift Recommendation/Gift-Recommendation/web_scraping/reviews/"
if not os.path.exists(reviews_folder):
    os.mkdir(reviews_folder)
    

In [ ]:
reviews = scrape.Sephora()

product_ids = df_products.product_id.unique()

for id_ in tqdm(product_ids):
    reviews.get_reviews(id_, reviews_folder)

reviews.browser.quit()

del reviews, product_ids

In [12]:
pwd

'/Users/valmadrid/DataScienceBootcamp/Projects/Gift Recommendation/Gift-Recommendation/web_scraping'

In [13]:
df = pd.read_csv("/Users/valmadrid/DataScienceBootcamp/Projects/Gift Recommendation/Gift-Recommendation/final_products.csv")
df.shape

(9300, 7)

In [16]:
ids = list(df.product_ref.unique())
ids = [id_ for id_ in ids if id_ not in df_products.product_id.unique()]
ids

['P457843',
 'P395205',
 'P451668',
 'P388509',
 'P393847',
 'P396312',
 'P396310',
 'P388507',
 'P393849',
 'P428827',
 'P377672',
 'P305801',
 'P420409',
 'P447585',
 'P456362',
 'P449348',
 'P433475',
 'P396821',
 'P400888',
 'P445416',
 'P401774',
 'P424981',
 'P428818',
 'P388629',
 'P456330',
 'P454933',
 'P453807',
 'P456430',
 'P453926',
 'P457688',
 'P455621',
 'P457690',
 'P457691',
 'P457689',
 'P394579',
 'P413261',
 'P381209',
 'P459405',
 'P427396',
 'P450079',
 'P434354',
 'P456561',
 'P429538',
 'P407189',
 'P385669',
 'P397883',
 'P422690',
 'P410768',
 'P412366',
 'P442553',
 'P435420',
 'P433808',
 'P446941',
 'P454540',
 'P436153',
 'P424946',
 'P384404',
 'P455015',
 'P442556',
 'P446355',
 'P442777',
 'P398803',
 'P392352',
 'P439432',
 'P449154',
 'P409044',
 'P451615',
 'P450869',
 'P440450',
 'P404931',
 'P392351',
 'P427901',
 'P435937',
 'P405436',
 'P437636',
 'P457847',
 'P458228',
 'P429927',
 'P432658',
 'P449598',
 'P451138',
 'P452734',
 'P450086',
 'P4

In [17]:
len(ids)

123

In [18]:
ids2 = list(df_products.product_id.unique())
ids2 = [id_ for id_ in ids2 if id_ not in df.product_ref.unique()]
ids2

['P460780',
 'P459233',
 'P457651',
 'P460563',
 'P457882',
 'P225362',
 'P457885',
 'P457886',
 'P460779',
 'P460566',
 'P460852',
 'P460851',
 'P460569',
 'P460847',
 'P460848',
 'P460567',
 'P460849',
 'P460850',
 'P460568',
 'P460565',
 'P404736',
 'P446867',
 'P433013',
 'P460557',
 'P460742',
 'P460740',
 'P460739',
 'P460560',
 'P458584',
 'P458583',
 'P436642',
 'P460515',
 'P421720',
 'P421719',
 'P460732',
 'P461800',
 'P459503',
 'P459505',
 'P458787',
 'P458790',
 'P9214873437',
 'P460743',
 'P440264',
 'P428119',
 'P460511',
 'P384507',
 'P460510',
 'P421285',
 'P459502',
 'P460728',
 'P460514',
 'P460513',
 'P425931',
 'P425932',
 'P380676',
 'P380677',
 'P442776',
 'P460730',
 'P460731',
 'P460729',
 'P459150',
 'P405108',
 'P460738',
 'P411830',
 'P389051',
 'P459011',
 'P460509',
 'P459838',
 'P419283',
 'P419284',
 'P418015',
 'P460784',
 'P459261',
 'P427610',
 'P414705',
 'P432085',
 'P428692',
 'P460717',
 'P414234',
 'P439907',
 'P460716',
 'P460719',
 'P410146',


In [19]:
len(ids2)

107